## Import Packages

In [2]:
!pip install gurobipy>=9.5.1

zsh:1: 9.5.1 not found


In [3]:
import gurobipy as gp
from gurobipy import GRB as GRB
import sys
sys.path.insert(0, "../utils")

import utils_functions
from unavailability import chairs_est_unavailability, members_est_unavailability

## Column Generation

In [4]:
chair_unavailability = utils_functions.calculate_unavailability(chairs_est_unavailability)
member_unavailability = utils_functions.calculate_unavailability(members_est_unavailability)

In [23]:
cleaned_chair_availability = []
for chair in chair_unavailability:
    cleaned_timeslots = [0] * 170
    for timeslot in chair:
        cleaned_timeslots[timeslot - 1] = 1
    cleaned_chair_availability.append(cleaned_timeslots)

cleaned_member_availability = []
for member in member_unavailability:
    cleaned_timeslots = [0] * 170
    for timeslot in member:
        cleaned_timeslots[timeslot - 1] = 1
    cleaned_member_availability.append(cleaned_timeslots)

## Optimization Model

In [24]:
# Setup Gurobi key
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '453f2da9-b8f2-448f-9f06-48081a9c3dc9')
e.setParam('WLSSECRET', 'd5675c81-7058-45f9-b6f2-30ca3855ed22')
e.setParam('LICENSEID', 868440)
e.start()

# Create the model within the Gurobi environment. This environment will be passed in as a parameter into all subsequent models
model = gp.Model(env=e)
model.setParam('Seed', 435)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868440
Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Set parameter Seed to value 435


In [25]:
# Sets
G = range(1, 2 + 1)
C = range(1, 10 + 1)
M = range(1, 30 + 1)
T = range(1, 170 + 1)

In [26]:
# Parameters
b = cleaned_chair_availability
a = cleaned_member_availability
NC = 5
NM = 15
R = 5

In [27]:
# Decision Varbiables
x = model.addVars(C, G, vtype=GRB.BINARY, name="x")
y = model.addVars(M, G, vtype=GRB.BINARY, name="y")

In [28]:
# Objective Value
model.setObjective(sum((sum(b[c-1][t-1] * x[c, g] for c in C) + sum(a[m-1][t-1] * y[m, g] for m in M)) for t in T for g in G), sense=GRB.MINIMIZE)

In [29]:
# Constraints
model.addConstrs(sum(x[c, g] for g in G) == 1 for c in C)
model.addConstrs(sum(y[m, g] for g in G) == 1 for m in M)
model.addConstrs(sum(x[c, g] for c in C) == NC for g in G)
model.addConstrs(sum(y[m, g] for m in M) == NM for g in G)
# model.addConstrs(R * sum(x[c, g] for c in C) == sum(y[m, g] for m in M) for g in G)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [30]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Optimize a model with 44 rows, 80 columns and 160 nonzeros
Model fingerprint: 0xc0b88719
Variable types: 0 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 1183.0000000
Presolve removed 44 rows and 80 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1183 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.183000000000e+03, best bound 1.183000000000e+03, gap 0.0000%


In [31]:
for g in G:
  print(f"Group {g}:")
  chairs = [c for c in C if x[c, g].X == 1]
  members = [m for m in M if y[m, g].X == 1]
  print(f"\tChairs: {chairs}")
  print(f"\tMembers: {members}")

Group 1:
	Chairs: [2, 7, 8, 9, 10]
	Members: [6, 7, 8, 9, 10, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Group 2:
	Chairs: [1, 3, 4, 5, 6]
	Members: [1, 2, 3, 4, 5, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
